In [1]:
import numpy as np                     # for random number generation
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
"""
Generated a set of fake data of value of a fund index with 365 daily quote each year, 
from date 2015.12.31 to 2036.12.31. The value of fund index as of 2015.12.31 is 100.00.
The values van be generated randomly but:
(Fund A,1) During 2016.01.01 to 2025.12.31 the average annual return is 15%
    and the volatility, standard deviation of annual return is 18%
(Fund A,2) During 2026.01.01 to 2036.12.31 the average annual return is 10%
    and the volatility, standard deviation of annual return is 11%
(Fund B,1) During 2016.01.01 to 2025.12.31 the average annual return is 10%
    and the volatility, standard deviation of annual return is 12%
(Fund B,2) During 2026.01.01 to 2036.12.31 the average annual return is 8%
    and the volatility, standard deviation of annual return is 10%
(Fund C,1) During 2016.01.01 to 2025.12.31 the average annual return is 18%
    and the volatility, standard deviation of annual return is 22%
(Fund C,2) During 2026.01.01 to 2036.12.31 the average annual return is 15%
    and the volatility, standard deviation of annual return is 18%

"""
# Generate date range from 2015-12-31 to 2036-12-31 (includes leap days automatically)

# ───────────────────────────────────────────────────────────────

import numpy as np
import pandas as pd
from datetime import datetime

# ───────────────────────────────────────────────────────────────
# CONFIGURATION
# ───────────────────────────────────────────────────────────────

START_DATE = datetime(2015, 12, 31)
END_DATE   = datetime(2036, 12, 31)
BASE_VALUE = 100.00
CHANGE_DATE = datetime(2026, 1, 1)

fund_configs = {
    'A': {
        '2016-2025': {'return': 0.15, 'vol': 0.18},
        '2026-2036': {'return': 0.10, 'vol': 0.11}
    },
    'B': {
        '2016-2025': {'return': 0.10, 'vol': 0.12},
        '2026-2036': {'return': 0.08, 'vol': 0.10}
    },
    'C': {
        '2016-2025': {'return': 0.18, 'vol': 0.22},
        '2026-2036': {'return': 0.15, 'vol': 0.18}
    },
    'D': {
        'whole_period': {'return': 0.03, 'vol': 0.007}
    }
}

# ───────────────────────────────────────────────────────────────
# Generate data
# ───────────────────────────────────────────────────────────────

dates = pd.date_range(START_DATE, END_DATE, freq='D')
prices = {}
np.random.seed(42)  # same numbers every time you run

for fund_name, config in fund_configs.items():
    price_series = pd.Series(index=dates, dtype=float)
    price_series.iloc[0] = BASE_VALUE

    if fund_name == 'D':
        r = config['whole_period']['return']
        v = config['whole_period']['vol']
        mu_annual = np.log(1 + r) - 0.5 * v**2
        mu_daily = mu_annual / 365
        sigma_daily = v / np.sqrt(365)

        for i in range(1, len(dates)):
            daily_log_ret = np.random.normal(mu_daily, sigma_daily)
            daily_log_ret = max(daily_log_ret, 0)  # no negative daily returns
            price_series.iloc[i] = price_series.iloc[i-1] * np.exp(daily_log_ret)
    else:
        regimes = {}
        for period, params in config.items():
            r = params['return']
            v = params['vol']
            mu_annual = np.log(1 + r) - 0.5 * v**2
            regimes[period] = {
                'mu_daily': mu_annual / 365,
                'sigma_daily': v / np.sqrt(365)
            }

        for i in range(1, len(dates)):
            dt = dates[i]
            if dt < CHANGE_DATE:
                mu_d = regimes['2016-2025']['mu_daily']
                sigma_d = regimes['2016-2025']['sigma_daily']
            else:
                mu_d = regimes['2026-2036']['mu_daily']
                sigma_d = regimes['2026-2036']['sigma_daily']
            
            log_ret = np.random.normal(mu_d, sigma_d)
            price_series.iloc[i] = price_series.iloc[i-1] * np.exp(log_ret)

    prices[fund_name] = price_series

# ───────────────────────────────────────────────────────────────
# Export to one CSV file (all four funds)
# ───────────────────────────────────────────────────────────────

df = pd.DataFrame({
    'Date': dates,
    'Fund_A': prices['A'],
    'Fund_B': prices['B'],
    'Fund_C': prices['C'],
    'Fund_D': prices['D']
})

df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
df = df.round(2)  # nice clean numbers like 829.33

filename = "funds_2015_to_2036.csv"
df.to_csv(filename, index=False)

print(f"Done! Exported all four funds to: {filename}")
print(f"Total number of rows: {len(df)} (daily data from 2015-12-31 to 2036-12-31)")
print("You can now open this file in Excel, Google Sheets, etc.")

Done! Exported all four funds to: funds_2015_to_2036.csv
Total number of rows: 7672 (daily data from 2015-12-31 to 2036-12-31)
You can now open this file in Excel, Google Sheets, etc.
